In [1]:
import torch
import pypose as pp

# 1. Intialization

In [2]:
x = pp.identity_SE3(2,1)
y = pp.randn_se3(2,2)
x.shape, x.gshape, x.gview(2), y

(torch.Size([2, 1, 7]),
 torch.Size([2, 1]),
 SE3Type Group:
 tensor([[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.]]),
 se3Type Group:
 tensor([[[ 0.0974, -0.2853,  0.4504, -1.0356, -1.2460,  0.9960],
          [-0.7348, -0.4837,  1.4968,  1.0494, -0.5384,  0.0581]],
 
         [[-1.2987,  0.6818,  0.3893, -1.0907,  0.8857, -1.7561],
          [ 1.8152, -0.3177, -0.9430, -0.3018, -0.2136,  0.7108]]]))

### All arguments in PyTorch are supported

In [3]:
a = pp.randn_SO3(3, device="cuda:0", dtype=torch.double, requires_grad=True)
b = pp.identity_like(a, device="cpu")
a, b

(SO3Type Group:
 tensor([[ 0.1040,  0.6170, -0.0862,  0.7753],
         [-0.6747,  0.4758, -0.4921,  0.2762],
         [-0.5743,  0.2541,  0.7776, -0.0299]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 SO3Type Group:
 tensor([[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]]))

# 2. Basic Operations

In [4]:
(x * y.Exp()).Log()

se3Type Group:
tensor([[[ 0.0974, -0.2853,  0.4504, -1.0356, -1.2460,  0.9960],
         [-0.7348, -0.4837,  1.4968,  1.0494, -0.5384,  0.0581]],

        [[-1.2987,  0.6818,  0.3893, -1.0907,  0.8857, -1.7561],
         [ 1.8152, -0.3177, -0.9430, -0.3018, -0.2136,  0.7108]]])

# 3. Adjoint Transforms

In [5]:
X = pp.randn_Sim3(6, dtype=torch.double)
a = pp.randn_sim3(6, dtype=torch.double)
b = X.AdjT(a)
print((X * b.Exp() - a.Exp() * X).abs().mean() < 1e-7)

tensor(True)


In [6]:
X = pp.randn_SE3(8)
a = pp.randn_se3(8)
b = X.Adj(a)
print((b.Exp() * X - X * a.Exp()).abs().mean() < 1e-7)

tensor(False)


# 4. Grdients

In [7]:
x = pp.randn_so3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf
loss = (x.Exp().Log()**2).sin().sum() # Just test, No phisical meaning
loss.backward()
loss, x.grad

(tensor(0.0848, device='cuda:0', grad_fn=<AliasBackward0>),
 tensor([[-0.0391, -0.1069,  0.1934],
         [ 0.3660,  0.0102, -0.2539],
         [-0.2846,  0.0647,  0.0703]], device='cuda:0'))

# 5. Test a Module

In [8]:
from torch import nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class TestNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.weight = pp.Parameter(pp.randn_so3(n))

    def forward(self, x):
        return self.weight.Exp() * x


n,epoch = 4, 5
net = TestNet(n).cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4], gamma=0.5)

print(net.weight)
for i in range(epoch):
    optimizer.zero_grad()
    inputs = pp.randn_SO3(n).cuda()
    outputs = net(inputs)
    loss = outputs.abs().sum()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(loss)

print("Parameter:", count_parameters(net))
print(net.weight)

so3Type Group:
Parameter containing:
tensor([[-1.4291, -0.4316, -0.8247],
        [ 0.3231,  0.7220,  0.5205],
        [-0.8938, -0.3425,  0.2319],
        [-0.7705,  2.4652, -1.5557]], device='cuda:0', requires_grad=True)
tensor(6.8381, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.0205, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.2309, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.7122, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.1693, device='cuda:0', grad_fn=<AliasBackward0>)
Parameter: 12
so3Type Group:
Parameter containing:
tensor([[-0.2157,  0.3625, -0.0970],
        [-0.4382,  0.3598, -0.5673],
        [-0.6617,  0.7419,  0.3041],
        [-0.8341,  1.1444, -0.7140]], device='cuda:0', requires_grad=True)
